<a href="https://colab.research.google.com/github/UltimateWing/WUG.Python.ShioajiApiLogic/blob/main/ShioajiApiFunc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install shioaji

In [54]:
import os

In [55]:
file = 'login.ini'
txtDict = {}
if os.path.isfile(file):
  print('Can get!!')
  with open(file, 'r') as f:
    t = f.read()
    onelinelist = t.split('\n')
    for i in onelinelist:
      kv = i.split(':')
      txtDict[kv[0]]=kv[1]
else:
  print('Can not get...')

for key in txtDict:
  #print('key=',key,',value=',txtDict[key])
  print('key=',key)

Can get!!
key= acc
key= pwd
key= caPath
key= caPwd


In [56]:
import shioaji as sj

In [77]:
class ShiojiApiLogic():
  #建構式
  def __init__(self, isSimulation: bool = True):
    self.IsSimulation = isSimulation
    if isSimulation:
      self.Api = sj.Shioaji(simulation = True)
    else:
      self.Api = sj.Shioaji()
      self.LoginId = ""
      self.IsConnected = False
      self.AccountList = []
      self.ReplyDict = {}
    
  #模擬環境初始化
  def InitSimulation(self, virtualIdNum: int = 1):
    virtualId = "PAPIUSER0" + str(virtualIdNum)
    pwd = "2222"
    self.Init(virtualId, pwd)

  #正式環境初始化
  def Init(self, loginId: str, loginPwd: str, caPath: str = "", caPwd: str = ""):
    self.Api.login(
        person_id = loginId,
        passwd = loginPwd,
        contracts_cb = lambda security_type: print(f"{repr(security_type)} fetch done.")
        )
    self.AccountList = self.Api.list_accounts()
    #註冊回報事件
    self.Api.set_order_callback(self.__placeCallback)    
    if self.IsSimulation == False:
      result = self.Api.activate_ca(
          ca_path = caPath,
          ca_passwd = caPwd,
          person_id = loginId
          )
      
  #委託下單
  def SendOrder(
      self, symbolId: str, orderPrice: float, orderQty: int, 
      bsType: str, priceType: str, timeInForce: str,
      sessionType: str, isFirstSell: bool, accountType: str):
    #註冊回報事件
    #self.Api.set_order_callback(self.__placeCallback)
    contract = self.__getContract(symbolId, accountType)    
    #使用預設帳號下單
    order = self.Api.Order(
        price = orderPrice,
        quantity = orderQty,
        action = self.__getAction(bsType),
        price_type = self.__getPriceType(priceType),
        order_type = self.__getTimeInForce(timeInForce),                      
        order_lot = self.__getSessionType(sessionType),
        first_sell = self.__getFirstSellType(isFirstSell),
        account = self.__getAccountInfo(accountType)
        )
    trade = self.Api.place_order(contract, order)
    return trade
    
  #def CancelOrder(self, orderNo: str)
    
    
  #登出
  def Logout(self):
    self.Api.logout()
    
    
  #Private Method
    
  #取得買賣類型
  def __getAction(self, bsType: str):
    if bsType == "Buy":
      return sj.constant.Action.Buy
    elif bsType == "Sell":
      return sj.constant.Action.Sell
    else:
      raise Exception(
          "Not correct Action: " + bsType + 
          ", only can input {Buy: 買, Sell: 賣}"
          )
            
  #取得價格類型
  def __getPriceType(self, priceType: str):
    if priceType == "Limit": #限價
      return sj.constant.StockPriceType.LMT
    elif priceType == "Market": #市價
      return sj.constant.StockPriceType.MKT
    elif priceType == "RangeMarket": #範圍市價
      return sj.constant.StockPriceType.MKP
    else:
      raise Exception(
          "Not correct StockPriceType: " + priceType + 
          ", only can input {Limit: 限價, Market: 市價, RangeMarket: 範圍市價}")

  #取得委託單時效性
  def __getTimeInForce(self, timeInForce: str):
    if timeInForce == "ROD":
      return sj.constant.TFTOrderType.ROD
    elif timeInForce == "IOC":
      return sj.constant.TFTOrderType.IOC
    elif timeInForce == "FOK":
      return sj.constant.TFTOrderType.FOK
    else:
      raise Exception(
          "Not correct TimeInForce: " + timeInForce +
          ", only can input {ROD, IOC, FOK}")
            
  #取得委託單盤別
  def __getSessionType(self, sessionType: str):
    if sessionType == "RegularTrading":
      return sj.constant.TFTStockOrderLot.Common
    elif sessionType == "AfterHoursTrading":
      return sj.constant.TFTStockOrderLot.Fixing
    elif sessionType == "OddLot":
      return sj.constant.TFTStockOrderLot.Odd
    elif sessionType == "OddLotAtSession":
      return sj.constant.TFTStockOrderLot.IntradayOdd
    else:
      raise Exception(
          "Not correct SessionType: " + sessionType +
          ", only can input {RegularTrading: 整股, " +
                     "AfterHoursTrading: 定盤, " +
                     "OddLot: 盤後零股, " + 
                     "OddLotAtSession: 盤中零股}"
          )

  def __getFirstSellType(self, isFirstSell: bool):
    if isFirstSell:
      return "true"
    else:
      return "false"

  #取得帳號資訊  
  def __getAccountInfo(self, accountType):
    if accountType == "Stk":
      return self.Api.stock_account
    elif accountType == "Fut":
      return self.Api.futopt_account
    else:
      raise Exception(
          "Not correct AccountType: " + accountType +
          ", only can input {Stk: 現貨帳戶資訊, Fut: 期貨帳戶資訊}"
      )

  #依帳戶類別取得商品合約，帳戶有股票和期貨，
  #以此為依據去取得Shioaji資料庫中Contracts.Stocks或Contracts.Futures的合約資料結構
  def __getContract(self, symbolId: str, accountType: str):
    if accountType == "Stk":
      return self.Api.Contracts.Stocks[symbolId]
    elif accountType == "Fut":
      return self.Api.Contracts.Futures[symbolId]
    else:
      raise Exception(
          "Not correct AccountType:" + accountType + 
          ", and then cause cannot catch contract. " +
          "Only can input {Stk: 現貨帳戶資訊, Fut: 期貨帳戶資訊}")
    
  #Event
    
  #接收委託回報
  def __placeCallback(self, stat, msg):
    print('my_place_callback')
    print('stat=', stat)
    print('----------')
    print('msg=',msg)        
    #if msg['order']['id']

In [ ]:
#正式登入
Api = ShiojiApiLogic(isSimulation=False)
Api.Init(txtDict['acc'], txtDict['pwd'], txtDict['caPath'], txtDict['caPwd'])

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.


In [72]:
#測試執行
Api = ShiojiApiLogic(isSimulation=True)
Api.InitSimulation(1)

Response Code: 0 | Event Code: 0 | Info: host '218.32.76.102:80', IP 218.32.76.102:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.


In [73]:
Api.AccountList

[FutureAccount(person_id='QBCCAIGJBJ', broker_id='F002000', account_id='9100020', signed=True, username='PAPIUSER01'),
 StockAccount(person_id='QBCCAIGJBJ', broker_id='9A95', account_id='0504350', signed=True, username='PAPIUSER01')]

In [76]:
trade = Api.SendOrder("2886", 30, 1, "Buy", "Limit", "ROD", "RegularTrading", False, "Stk")

In [70]:
Api.Logout()